# Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Цель проекта
    
    Выбор региона для  добычи нефти с максимальной прибылью с учетом того, что:
    - известны показатели для оценки скважины, позволяющие предсказать возможный объем месторождения
    - всего регионов три
    - в каждом 10 000 месторождений
    - бюджет на разработку региона 10 млрд, а разработка одной скважины стоит 50 млн
    - баррель нефти приносит доход 4500
    - риск убытков должен быть не выше 2.5%.
    

# 1. Загрузка и подготовка данных

In [1]:
import pandas as pd

In [2]:
import matplotlib.pyplot as plt

In [3]:
from sklearn.linear_model import LinearRegression

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
from sklearn.metrics import mean_squared_error

In [6]:
import numpy as np

In [7]:
from scipy import stats as st

 # Описание данных
Данные геологоразведки трёх регионов находятся в файлах: geo_data_0.csv, geo_data_1.csv и geo_data_2.csv.
- id — уникальный идентификатор месторождения;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в месторождении (тыс. баррелей).

In [8]:
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv') #данные для первого региона

In [9]:
geo_data_0.head(40)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


In [10]:
geo_data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [11]:
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv') #данные для второго региона

In [12]:
geo_data_1.head(40)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


In [13]:
geo_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [14]:
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv') #данные для третьего региона

In [15]:
geo_data_2.head(40)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


In [16]:
geo_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Нормируемыми показателями качества товарной нефти являются:

- содержание в ней воды;
- количество примесей механического характера;
- показатель давления насыщенных нефтяных паров;
- содержание солей хлористого вида;
- содержание хлор – органических химических соединений.

Эти показатели не могут быть отрицательными, следовательно, нужно взять их модули (вероятно, внесли ошибочно отрицательные значения).
Также переведем все названия в нижний регистр и проверим на дубликаты.

Напишем функцию, т.к. у нас 3 датасета.

In [17]:
def predobrab(data):
    data[['f0', 'f1', 'f2', 'product']] = data[['f0', 'f1', 'f2', 'product']].abs()
    data['id'] = data['id'].str.lower()
    return data

In [18]:
geo_data_0 = predobrab(geo_data_0) # проверяем новый вид датасета, а также количество дубликатов (аналогично - для двух других датасетов) 
print(geo_data_0.head(),'\n', 'Количество дубликатов:', geo_data_0.duplicated().sum())

      id        f0        f1        f2     product
0  txeyh  0.705745  0.497823  1.221170  105.280062
1  2acmu  1.334711  0.340164  4.365080   73.037750
2  409wp  1.022732  0.151990  1.419926   85.265647
3  ijlyr  0.032172  0.139033  2.978566  168.620776
4  xdl7t  1.988431  0.155413  4.751769  154.036647 
 Количество дубликатов: 0


In [19]:
geo_data_1 = predobrab(geo_data_1) 
print(geo_data_1.head(),'\n', 'Количество дубликатов:', geo_data_1.duplicated().sum())

      id         f0         f1        f2     product
0  kbedx  15.001348   8.276000  0.005876    3.179103
1  62mp7  14.272088   3.475083  0.999183   26.953261
2  vye1p   6.263187   5.948386  5.001160  134.766305
3  kcrkz  13.081196  11.506057  4.999415  137.945408
4  ahl4o  12.702195   8.147433  5.004363  134.766305 
 Количество дубликатов: 0


In [20]:
geo_data_2 = predobrab(geo_data_2) 
print(geo_data_2.head(),'\n', 'Количество дубликатов:', geo_data_2.duplicated().sum())

      id        f0        f1        f2     product
0  fwxo0  1.146987  0.963328  0.828965   27.758673
1  wjtft  0.262778  0.269839  2.530187   56.069697
2  ovluw  0.194587  0.289035  5.586433   62.871910
3  q6ca6  2.236060  0.553760  0.930038  114.572842
4  wpmux  0.515993  1.716266  5.899011  149.600746 
 Количество дубликатов: 0


# 2. Обучение и проверка модели

Предполагаем, что product (объём запасов в месторождения) зависит от трех показателей(f0, f1, f2).
В качестве модели используется линейная регрессия (не склонна к переобучению из-за малого количества параметров). Работать будем со всеми 100 000 месторождениями (чем больше значений, тем лучше качество модели).

Разделяем исходные данные на обучающую и валидационную выборки в соотношении 75:25.
Напишем функцию, т.к. у нас 3 датасета.

In [27]:
def data_split(data): # функция разделения данных на обучающую и валидационную выборки 
    target = data['product']
    features = data.drop(columns=['product', 'id'] , axis=1)
    #Чтобы не мешать модели, удалим и столбец, который не несет нужной нам информации.
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    return features_train, features_valid, target_train, target_valid

In [28]:
features_train0, features_valid0, target_train0, target_valid0 = data_split(geo_data_0)

In [29]:
features_train1, features_valid1, target_train1, target_valid1 = data_split(geo_data_1)

In [30]:
features_train2, features_valid2, target_train2, target_valid2 = data_split(geo_data_2)

In [31]:
def model(features_train, features_valid, target_train, target_valid):
# функция для работы модели линейной регрессии, ниже выведем на экран для трех регионов показатели
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    mse = mean_squared_error(target_valid, predictions_valid)
    rmse = mse ** 0.5
    product_mean = predictions_valid.mean()
    return product_mean, rmse

In [32]:
print(' Cредний запас сырья и RMSE модели для датасета geo_data_0', model(features_train0, features_valid0, target_train0, target_valid0), 'соответственно.','\n', 'Cредний запас сырья и RMSE модели для датасета geo_data_1', model(features_train1, features_valid1, target_train1, target_valid1), 'соответственно.', '\n', 'Cредний запас сырья и RMSE модели для датасета geo_data_2', model(features_train2, features_valid2, target_train2, target_valid2), 'соответственно.')

 Cредний запас сырья и RMSE модели для датасета geo_data_0 (92.51881935132751, 40.3427566762566) соответственно. 
 Cредний запас сырья и RMSE модели для датасета geo_data_1 (68.73031469768347, 1.5360594290776344) соответственно. 
 Cредний запас сырья и RMSE модели для датасета geo_data_2 (95.03696840612521, 40.615887208944) соответственно.


In [34]:
def predictions(features_train, features_valid, target_train, target_valid):
# функция для создания списка с предсказаниями объемов запасов
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    return predictions_valid

In [35]:
predictions_valid0 = predictions(features_train0, features_valid0, target_train0, target_valid0)

In [36]:
predictions_valid2 = predictions(features_train2, features_valid2, target_train2, target_valid2)

In [37]:
geo_data_0['predictions'] = pd.DataFrame(predictions_valid0 ) #добавляем столбец с предсказаниями для двух таблиц отобранных регионов

In [38]:
geo_data_2['predictions'] = pd.DataFrame(predictions_valid2 )

In [39]:
geo_data_0.head()

,id,f0,f1,f2,product,predictions
0,txeyh,0.705745,0.497823,1.221170,105.280062,91.641670
1,2acmu,1.334711,0.340164,4.365080,73.037750,75.046616
2,409wp,1.022732,0.151990,1.419926,85.265647,81.217538
3,ijlyr,0.032172,0.139033,2.978566,168.620776,73.260562
4,xdl7t,1.988431,0.155413,4.751769,154.036647,64.032154


In [40]:
geo_data_0 = geo_data_0.dropna() 
# т.к. предсказания были только для части датасетов, удаляем строки нез предсказаний. У нас останется 25000 строк - это достаточно для оценки прибыли.

In [41]:
geo_data_2 = geo_data_2.dropna()

In [42]:
geo_data_0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25000 entries, 0 to 24999
Data columns (total 6 columns):
id             25000 non-null object
f0             25000 non-null float64
f1             25000 non-null float64
f2             25000 non-null float64
product        25000 non-null float64
predictions    25000 non-null float64
dtypes: float64(5), object(1)
memory usage: 1.3+ MB


# 3. Подготовка к расчёту прибыли

Для расчета прибыли нам понабятся следующие данные (константы Python):

In [49]:
n = 500 # количество точек для исследования
b = 10000000000 # бюджет на разработку месторождений 
w = 50000000 # стоимость бурения одной скважины
br = 4500 # прибыль одного барреля сырья 

Бюджет на разработку месторождений 10 млрд.
Чтобы получить выгоду, нам необходимо выручить больше данной суммы с одного месторождения.

Исходя из бюджета на разработку месторождений — 10 млрд рублей и стоимость бурения одной скважины — 50 млн рублей, на этот бюджет мы сможем пробурить 200 скважин (test_n , далее будем использовать эту цифру для выборки).

Посчитаем:
    - 10000000000 / 4500 = 2 222 222 - суммарное искомое кол-во баррелей, которые нужно получить (минимальный средний объём сырья в месторождениях региона, достаточный для его разработки)
    - 2 222 222 / 200 = 11111 - минимальное среднее количество барелей, которое должна принести каждая скважина.
    
В отобранных регионах мы имеем средний объём сырья для одной скважины для geo_data_0 -  92519 баррелей и для geo_data_2 -  95037 баррелей.

Этого объема должно быть достаточно.

In [50]:
test_n = 200 # Максимальное кол-во скважин, которое можно пробурить, исходя их бюджета

In [51]:
def revenue(probabilities): # функция для расчёта прибыли по набору отобранных месторождений и предсказаний модели
    return test_n * br * probabilities*1000 - b

# 4. Расчёт прибыли и рисков 

In [52]:
state = np.random.RandomState(12345)

In [53]:
def camputation(data):
# функция для расчета средняя прибыли и 95%-й доверительный интервала с использование техники Bootstrap с 1000 выборок, чтобы найти распределение прибыли 
    values = []

    for i in range(1000):
        target_subsample = data['predictions'].sample(n=200, replace=True, random_state=state)
        values.append(revenue(target_subsample.mean()*1000))

    values = pd.Series(values)

    mean = values.mean()
    confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem())
    return  mean, confidence_interval

In [54]:
print(' Cредняя прибыль и 95%-й доверительный интервал для первого региона:', '\n', camputation(geo_data_0), '\n','Cредняя прибыль и 95%-й доверительный интервал для третьего региона:', '\n', camputation(geo_data_2))

 Cредняя прибыль и 95%-й доверительный интервал для первого региона: 
 (83255857009041.25, (83183632390049.83, 83328081628032.67)) 
 Cредняя прибыль и 95%-й доверительный интервал для третьего региона: 
 (85508458203597.25, (85434277177267.1, 85582639229927.4))


✔Вывод: средняя прибыль третьего региона больше первого на 2,7% или 2285 млрд.

In [55]:
def rev_values(data): # функция для создания списка средних значений прибыли
    values = []

    for i in range(1000):
        target_subsample = data['predictions'].sample(n=200, replace=True, random_state=state)
        values.append(revenue(target_subsample.mean()*1000))

    values = pd.Series(values)

    return  values

Напишем функцию для гипотезы: средняя прибыль меньше бюджета разработки (b=10 млрд),  альтернативная гипотеза - средняя прибыль не меньше бюджета разработки

In [56]:
def risks(data):
    alpha = 0.05 # критический уровень статистической значимости для оценки 2.5% риска

    # тест односторонний: p-value будет в два раза меньше
    
    results = st.ttest_1samp(rev_values(data), b) 
    pvalue = (results.pvalue) / 2
    
# отвергаем гипотезу и только если выборочное среднее значимо меньше бюджета

    if (results.pvalue < alpha) and (rev_values(data).mean() < b):
        answer = "Риск превышает 2.5%"
    else:
        answer = "Риск не превышает 2.5% "
    return answer

In [57]:
print(' Для первого региона:', '\n', risks(geo_data_0), '\n', 'Для третьего региона:', '\n', risks(geo_data_2)) 

 Для первого региона: 
 Риск не превышает 2.5%  
 Для третьего региона: 
 Риск не превышает 2.5% 


# Общие выводы


В задече нам требовалось выбрать регион для  добычи нефти с максимальной прибылью с учетом того, что:
- известны показатели для оценки скважины, позволяющие предсказать возможный объем месторождения
- всего регионов три
- в каждом 10 000 месторождений
- бюджет на разработку региона 10 млрд, а разработка одной скважины стоит 50 млн
- баррель нефти приносит доход 4500
- риск убытков должен быть не выше 2.5%.

В ходе проделанного исследования были выбраны 1 и 3 регионы месторождений для дальнейшего изучения (со средними запасами 92.52 и 95.04 соответственно), несмотря на значительно худшие показатели RMSE, чем во втором регионе (самуя точная модель (RMSE=1.54), однако среднее значение запасов тут самое малое (68.73)).

При расчетах получили следующие значения:

     Cредняя прибыль и 95%-й доверительный интервал для первого региона: 
     83233146701798.36, (83161609787138.92, 83304683616457.8)
     Cредняя прибыль и 95%-й доверительный интервал для третьего региона: 
     85518757154029.72, (85444970873780.17, 85592543434279.27)

Риск убытков не превышает 2.5% для обоих регионов.

При сравнении полученных данных прибыль третьего региона больше первого на 2,7% или 2285 млрд, поэтому для разработки предлагается третий регион.
